In [1]:
# import these
import qiskit
from qiskit import * 
from qiskit.tools import job_monitor
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import numpy as np
# ADD THE FOLLOWING LINE TO THE ACTUAL CODE
#from ipynb.fs.full.quantum_algs import run_program

# dont add, just expermenting rn
#from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter


In [2]:
def run_program(circuit, qubit_num, tries, type_of_comp = "" ):
    
    #helper function
    def get_jobs(b):
        return b.status().pending_jobs
    
    IBMQ.load_account()
    if(type_of_comp == "q_sim"):
        sim_backend = Aer.get_backend ("qasm_simulator")
        job = execute(circuit, backend = sim_backend, shots = tries)
        return job.result()
    
    if(type_of_comp == 'state_vec'):
        sim_backend = Aer.get_backend("statevector_simulator")
        return execute(circuit, backend = sim_backend).result().get_statevector()
    
    
    provider = IBMQ.get_provider(hub = "ibm-q")
    small_devices = provider.backends(filters=lambda x: x.configuration().n_qubits >= qubit_num
                                   and not x.configuration().simulator and x.status().operational==True)
    #print(f"{type(small_devices[0])} lol")
    #sorted list
    computer = sorted(small_devices, key = get_jobs)[0]
    
    print(f"{computer.name()}: {computer.status().pending_jobs} jobs")
    print(f"Available resources: {len(computer.properties().qubits)} qubits")
    job = execute(circuit, backend = computer, shots = tries)
    job_monitor(job)
    return job.result()
      #  pendJob = 999999999
     #   for backend in provider.backends():
   #         try: 
   #             qubit_count = len(backend.properties().qubits)
     #           if((backend.status().pending_jobs < pendJob) and (qubit_count >= qubit_num)):
    #                pendJob = backend.status().pending_jobs
     #               currentComp = backend.name()
      #              amount_of_qubits = qubit_count
       #     except:
       #         pass

    backend = provider.get_backend(currentComp)
    job = execute(circuit, backend = backend, shots = tries)
    job_monitor(job)
    return job.result()
        
    backend = provider.get_backend()
        
#    if(type_of_comp == 'mit'):
#       cal_circuits, state_labels - complete_meas_cal(qr = circuit.qregs[0], circlablel = 'calibration')
#        cal_job = execute(cal_circuits, backend = provider.get_backend('ibmqx2'), shots = tries, optimization_level = 0)
#        cal_results = cal_job.result()
#        meas_fitter = CompleteMeasFitter(cal_results, statelabels)
#        meas_filter = meas_fitter.filter
#        return meas_filter.apply(job.result)
    
    return job.result()